In [1]:
import pandas as pd
from pandas.tools.plotting import scatter_matrix
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cross_validation import ShuffleSplit
from sklearn.cross_validation import train_test_split
from sklearn import metrics as mt
import seaborn as sns

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier)
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve
from sklearn.pipeline import make_pipeline

c:\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
eopen=pd.read_csv('Data\email_opened_table.csv')
click=pd.read_csv('Data\link_clicked_table.csv')
email=pd.read_csv('Data\email_table.csv')

In [16]:
click['Clicked'] = 1
eopen['Opened'] = 1

* What percentage of users opened the email?
* what percentage clicked on the link within the email?

In [19]:
merged = email.merge(click, how='outer')
df = eopen.merge(merged, how='outer')
df.fillna(0)

In [33]:
print("Percentage of users opened the email {}%".format(df['Opened'].sum()*100/len(df)))
print("Percentage of clicked on the link within the email {}%".format(df['Clicked'].sum()*100/len(df)))

Percentage of users opened the email 10.345%
Percentage of clicked on the link within the email 2.119%
